<a href="https://colab.research.google.com/github/czasoprzestrzenny/public_repo/blob/master/new_data_source_coronavirus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#author czasoprzestrzenny 15.03.2020
#data source : https://data.humdata.org/dataset/5dff64bc-a671-48da-aa87-2ca40d7abf02
#Johns Hopkins School of Public Health

In [133]:
pip install folium

In [134]:
pip install chart_studio

In [0]:
import warnings
warnings.filterwarnings("ignore")
import folium as folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium import plugins
import datetime 
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from plotly.subplots import make_subplots


In [0]:
url_confirmed = """https://data.humdata.org/hxlproxy/data/download/time_series_covid19_confirmed_global_narrow.csv?dest=data_edit&filter01=explode&explode-header-att01=date&explode-value-att01=value&filter02=rename&rename-oldtag02=%23affected%2Bdate&rename-newtag02=%23date&rename-header02=Date&filter03=rename&rename-oldtag03=%23affected%2Bvalue&rename-newtag03=%23affected%2Binfected%2Bvalue%2Bnum&rename-header03=Value&filter04=clean&clean-date-tags04=%23date&filter05=sort&sort-tags05=%23date&sort-reverse05=on&filter06=sort&sort-tags06=%23country%2Bname%2C%23adm1%2Bname&tagger-match-all=on&tagger-default-tag=%23affected%2Blabel&tagger-01-header=province%2Fstate&tagger-01-tag=%23adm1%2Bname&tagger-02-header=country%2Fregion&tagger-02-tag=%23country%2Bname&tagger-03-header=lat&tagger-03-tag=%23geo%2Blat&tagger-04-header=long&tagger-04-tag=%23geo%2Blon&header-row=1&url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_confirmed_global.csv"""
read_confirmed=pd.read_csv(url_confirmed)
url_deaths = """https://data.humdata.org/hxlproxy/data/download/time_series_covid19_deaths_global_narrow.csv?dest=data_edit&filter01=explode&explode-header-att01=date&explode-value-att01=value&filter02=rename&rename-oldtag02=%23affected%2Bdate&rename-newtag02=%23date&rename-header02=Date&filter03=rename&rename-oldtag03=%23affected%2Bvalue&rename-newtag03=%23affected%2Binfected%2Bvalue%2Bnum&rename-header03=Value&filter04=clean&clean-date-tags04=%23date&filter05=sort&sort-tags05=%23date&sort-reverse05=on&filter06=sort&sort-tags06=%23country%2Bname%2C%23adm1%2Bname&tagger-match-all=on&tagger-default-tag=%23affected%2Blabel&tagger-01-header=province%2Fstate&tagger-01-tag=%23adm1%2Bname&tagger-02-header=country%2Fregion&tagger-02-tag=%23country%2Bname&tagger-03-header=lat&tagger-03-tag=%23geo%2Blat&tagger-04-header=long&tagger-04-tag=%23geo%2Blon&header-row=1&url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_deaths_global.csv"""
read_deaths=pd.read_csv(url_deaths)

In [0]:
data_c = read_confirmed.loc[1:]
data_d = read_deaths.loc[1:]

data_c.rename(columns={'Value':'total_cases'}, inplace=True)
data_d.rename(columns={'Value':'total_deaths'}, inplace=True)

In [0]:
def prepare_to_plot(locations,frame,column_on_x, column_on_y, chart_title, xaxis_title,  yaxis_title, chart_type = "Bar"):
  """prepare data to ploting"""
  data = []
  
  for i, item in enumerate(locations):
        
        next_loc = pd.DataFrame(frame[frame['Province/State'] == item])
        #print (item)

        if chart_type =="Scatter":
          trace = go.Scatter(
          x=next_loc[column_on_y],
          y=next_loc[column_on_x],
          name=str(item))
        elif chart_type =="Bar":
          trace = go.Bar(
          x=next_loc[column_on_y],
          y=next_loc[column_on_x],
          name=str(item))

        layout = dict(title= chart_title + ". Generated : " + str('{:%Y-%m-%d}'.format(datetime.datetime.now())),
                      xaxis=dict(title= column_on_y),
                      yaxis=dict(title='col'),
                      xaxis_title= xaxis_title,
                      yaxis_title= yaxis_title,
                      )

        data.append(trace)

  data = dict(data=data, layout=layout)
  return data

In [0]:
#data_c_r= data_c.merge(data_r, 'inner', on=['Country/Region', 'Date', 'Province/State'])
data_final = data_c.merge(data_d, 'inner', on=['Country/Region', 'Date', 'Province/State'])

In [0]:
#data = data_final[['Province/State', 'Country/Region', 'Lat_x', 'Long_x', 'Date', 'total_cases', 'total_recovered', 'total_deaths']]
data = data_final[['Province/State', 'Country/Region', 'Lat_x', 'Long_x', 'Date', 'total_cases', 'total_deaths']]
data['Province/State'] = data['Province/State'].where(pd.notnull, data['Country/Region'])
data.sort_values(['Province/State','Country/Region', 'Date'], ascending=False, inplace= True)
data["total_cases"] = data["total_cases"].apply(pd.to_numeric)
data[["Lat_x", "Long_x", "total_cases", "total_deaths"]] = data[["Lat_x", "Long_x", "total_cases","total_deaths"]].apply(pd.to_numeric)
data["Lat_x"] = data["Lat_x"].apply(pd.to_numeric)
data['Province/State']= data['Province/State'].astype(str) 
data['yesterday_cases'] = data.groupby(['Province/State','Country/Region'])['total_cases'].shift(-1)
data['yesterday_deaths'] = data.groupby(['Province/State','Country/Region'])['total_deaths'].shift(-1)
data['new_cases'] = data['total_cases'] - data['yesterday_cases']
data['new_deaths'] = data['total_deaths'] - data['yesterday_deaths']
#data[["new_cases", "total_deaths", "total_recovered"]] = data[["new_cases", "total_deaths","total_recovered"]].apply(pd.to_numeric)
data[["new_cases", "total_deaths"]] = data[["new_cases", "total_deaths"]].apply(pd.to_numeric)
data['PercentOfNew'] =  (data.total_cases / (data.total_cases - data.new_cases)) -1
data['FatalityRate'] =  data.total_deaths / data.total_cases
data['lat_long'] = list(zip(data.Lat_x, data.Long_x))
data['Country/Region']= data['Country/Region'].apply(lambda x:x.upper())
data['Province/State']= data['Province/State'].apply(lambda x:x.upper())
#[data[col].apply(lambda x:x.upper()) for col in ['Country/Region', 'Province/State']]
data['Date'] = pd.to_datetime(data['Date'])
#data.head(5)

In [0]:
final_frame_map = data.drop_duplicates(subset=['Province/State','Country/Region'], keep= 'first')

In [0]:
lat_long = final_frame_map.lat_long.values.tolist()
total_cases = final_frame_map.total_cases.values.tolist()
location = final_frame_map['Province/State'].unique().tolist()

In [0]:
def color_change(c):
    """ convert a points to colors"""
    if(c < 100):
        return('green')
    elif(100 <= c < 1000):
        return('orange')
    else:
        return('red')

In [176]:
map = folium.Map()

for point in range(0, len(lat_long)):

    folium.CircleMarker(lat_long[point], 
                        radius = 9,
                        #popup=str(total_cases[point]), 
                        tooltip = str(total_cases[point]),
                        fill_color = color_change(total_cases[point]),
                        fill_opacity = 0.6               
                        ).add_to(map)

plugins.MarkerCluster(lat_long).add_to(map)

#plugins.TimestampedGeoJson({
#    'type': 'FeatureCollection',
#}, period='PT1M', add_last_point=True).add_to(map)
map

In [0]:
## choose data to further analysis

In [0]:
location_eu = [loc for loc in location if loc in ['UNITED KINGDOM', 'ITALY', 'SPAIN', 'GERMANY', 'FRANCE', 'POLAND', \
                                                  'SLOVAKIA', 'UKRAINE','CZECH REPUBLIC']]
final_frame_to_chart = data[data['Date']> pd.Timestamp(2020, 3, 1)]
final_frame_to_chart['is_country'] = final_frame_to_chart['Province/State']==final_frame_to_chart['Country/Region']
final_frame_to_chart = final_frame_to_chart[['Province/State','Country/Region', 'Date', 'total_cases', 'total_deaths', 'new_cases', \
                                             'new_deaths', 'PercentOfNew', 'FatalityRate', 'is_country']]

In [179]:
pre_plot = prepare_to_plot(locations = location_eu, frame = final_frame_to_chart, 
                           column_on_x = 'total_cases', column_on_y = 'Date', 
                           chart_title = "COVID-19 / TOTAL_CASES for european countries", 
                           xaxis_title ="Date",  yaxis_title = "Number of cases", chart_type= 'Scatter')
 
iplot(pre_plot)

In [180]:
pre_plot = prepare_to_plot(locations = location_eu, frame = final_frame_to_chart, 
                           column_on_x = 'total_deaths', column_on_y = 'Date', 
                           chart_title = "COVID-19 / TOTAL_DEATHS for european countries", 
                           xaxis_title ="Date",  yaxis_title = "Number of cases", chart_type= 'Scatter')
 
iplot(pre_plot)

In [181]:
pre_plot = prepare_to_plot(locations = location_eu, frame = final_frame_to_chart, 
                           column_on_x = 'new_cases', column_on_y = 'Date', 
                           chart_title = "COVID-19 / NUMBER OF NEW_CASES (day by day) for european countries", 
                           xaxis_title ="Date",  yaxis_title = "% of new cases", chart_type= 'Bar')
 
iplot(pre_plot)

In [0]:
# the biggest number of total deaths

In [0]:
frame_new = final_frame_to_chart#[final_frame_to_chart['is_country']==False]
frame_new = frame_new.sort_values(['Date', 'total_deaths'], ascending=False)
location_new = frame_new['Province/State'].head(10).unique().tolist()
frame_new = frame_new[frame_new['Date']> pd.Timestamp(2020, 3, 7)]


In [206]:
fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(
    x=frame_new['Province/State'].head(10).values.tolist(), y=frame_new['total_cases'].values.tolist(), name='total_cases', marker_color='indianred',
    text=frame_new['total_cases'].values.tolist(), textposition='auto'), secondary_y=False)
fig.add_trace(go.Bar(
    x=frame_new['Province/State'].head(10).values.tolist(), y=frame_new['total_deaths'].values.tolist(), name='total_deaths', text=frame_new['total_deaths'].values.tolist(),
    textposition='auto', marker_color='lightsalmon'), secondary_y=False)

fig.update_layout(xaxis_tickangle=-45, title_text="Number of total deaths versus total cases ")
fig.show()

In [207]:
pre_plot = prepare_to_plot(locations = location_new, frame = frame_new, 
                           column_on_x = 'total_cases', column_on_y = 'Date', 
                           chart_title = "COVID-19 / TOTAL_CASES for location with highest value of total deaths", 
                           xaxis_title ="Date",  yaxis_title = "Number of cases", chart_type= 'Scatter')
 
iplot(pre_plot)

In [0]:
frame_new = final_frame_to_chart#[final_frame_to_chart['is_country']==False]
frame_new = frame_new.sort_values(['Date', 'new_cases'], ascending=False)
location_new = frame_new['Province/State'].head(10).unique().tolist()


In [187]:
pre_plot = prepare_to_plot(locations = location_new, frame = frame_new, 
                           column_on_x = 'total_cases', column_on_y = 'Date', 
                           chart_title = "COVID-19 / TOTAL_CASES for location with highest value of new cases", 
                           xaxis_title ="Date",  yaxis_title = "Number of cases", chart_type= 'Scatter')
 
iplot(pre_plot)

In [0]:
##the highest value of Death Rate

In [189]:
frame_new.sort_values(['Date','Province/State'], ascending=False, inplace= True)
final_frame_new_deaths = frame_new.drop_duplicates(subset=['Date','Province/State'], keep= 'first').sort_values(['Date','new_deaths'], ascending= False)
final_frame_new_deaths.head(10)

,Province/State,Country/Region,Date,total_cases,total_deaths,new_cases,new_deaths,PercentOfNew,FatalityRate,is_country
8946,ITALY,ITALY,2020-03-24,69176,6820,5249.0,743.0,0.082109,0.098589,True
13167,SPAIN,SPAIN,2020-03-24,39885,2808,4749.0,497.0,0.135161,0.070402,True
6993,FRANCE,FRANCE,2020-03-24,22304,1100,2448.0,240.0,0.123288,0.049319,True
14805,US,US,2020-03-24,53740,706,9893.0,149.0,0.225625,0.013137,True
8694,IRAN,IRAN,2020-03-24,24811,1934,1762.0,122.0,0.076446,0.077949,True
14301,UNITED KINGDOM,UNITED KINGDOM,2020-03-24,8077,422,1427.0,87.0,0.214586,0.052247,True
10962,NETHERLANDS,NETHERLANDS,2020-03-24,5560,276,811.0,63.0,0.170773,0.049640,True
7812,GERMANY,GERMANY,2020-03-24,32986,157,3930.0,34.0,0.135256,0.004760,True
1449,BELGIUM,BELGIUM,2020-03-24,4269,122,526.0,34.0,0.140529,0.028578,True
1827,BRAZIL,BRAZIL,2020-03-24,2247,46,323.0,12.0,0.167879,0.020472,True


In [190]:
frame_p2 = final_frame_new_deaths[final_frame_new_deaths['new_deaths'] >1]

pre_plot = prepare_to_plot(locations = frame_p2['Province/State'].head(5).values.tolist(), frame = frame_p2,
                           column_on_x = 'new_deaths', column_on_y = 'Date', 
                           chart_title = "COVID-19 / Number of deaths (day by day) for locations with highest number of new deaths", 
                           xaxis_title ="Date",  yaxis_title = "Number of cases", chart_type= 'Bar')
 
iplot(pre_plot)

In [0]:
# LOCATIONS with highest FATALITY RATE

In [192]:
frame_new.sort_values(['Date', 'FatalityRate'], ascending=False, inplace= True)
frame_new[(frame_new.total_deaths > 1) & (frame_new.new_deaths > 1)].head(5)

,Province/State,Country/Region,Date,total_cases,total_deaths,new_cases,new_deaths,PercentOfNew,FatalityRate,is_country
8946,ITALY,ITALY,2020-03-24,69176,6820,5249.0,743.0,0.082109,0.098589,True
8757,IRAQ,IRAQ,2020-03-24,316,27,50.0,4.0,0.187970,0.085443,True
8631,INDONESIA,INDONESIA,2020-03-24,686,55,107.0,6.0,0.184801,0.080175,True
8694,IRAN,IRAN,2020-03-24,24811,1934,1762.0,122.0,0.076446,0.077949,True
126,ALGERIA,ALGERIA,2020-03-24,264,19,34.0,2.0,0.147826,0.071970,True


In [0]:
# LOCATIONS with highest PERCENT OF NEW cases

In [194]:
frame_new.sort_values(['Date', 'PercentOfNew'], ascending=False, inplace= True)
frame_new[(frame_new.total_deaths > 1) & (frame_new.new_deaths > 1)].head(5)

,Province/State,Country/Region,Date,total_cases,total_deaths,new_cases,new_deaths,PercentOfNew,FatalityRate,is_country
12222,ROMANIA,ROMANIA,2020-03-24,794,11,218.0,4.0,0.378472,0.013854,True
378,ARGENTINA,ARGENTINA,2020-03-24,387,6,86.0,2.0,0.285714,0.015504,True
6300,DOMINICAN REPUBLIC,DOMINICAN REPUBLIC,2020-03-24,312,6,67.0,3.0,0.273469,0.019231,True
14805,US,US,2020-03-24,53740,706,9893.0,149.0,0.225625,0.013137,True
14049,TURKEY,TURKEY,2020-03-24,1872,44,343.0,7.0,0.224330,0.023504,True


In [195]:
new = data.groupby(['Date']).sum().reset_index()
new = new[new['Date']> pd.Timestamp(2020, 3, 1)]
new.tail()

,Date,Lat_x,Long_x,total_cases,total_deaths,yesterday_cases,yesterday_deaths,new_cases,new_deaths,PercentOfNew,FatalityRate
58,2020-03-20,5273.524133,6275.056743,271988,11299,242456.0,9867.0,29532.0,1432.0,inf,4.009156
59,2020-03-21,5273.524133,6275.056743,304348,12973,271988.0,11299.0,32360.0,1674.0,inf,4.105824
60,2020-03-22,5273.524133,6275.056743,336901,14651,304348.0,12973.0,32553.0,1678.0,inf,inf
61,2020-03-23,5273.524133,6275.056743,378176,16505,336901.0,14651.0,41275.0,1854.0,inf,inf
62,2020-03-24,5273.524133,6275.056743,417986,18625,378176.0,16505.0,39810.0,2120.0,inf,inf


In [210]:
dates = new['Date'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=dates, y=new['total_deaths'].values.tolist(), name="total_deaths"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=dates, y=new['total_cases'].values.tolist(), name="total_cases"),
    secondary_y=True,
)
fig.add_trace(
    go.Bar(x=dates, y=new['new_cases'].values.tolist(), name="new_cases"),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(x=dates, y=new['new_deaths'].values.tolist(), name="new_deaths"),
    secondary_y=False,
)
# Add figure title
fig.update_layout(
    title_text="World covid statistics"
)
# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
#fig.update_yaxes(title_text="<b>deaths - new cases</b>", secondary_y=False)
#fig.update_yaxes(title_text="<b>total cases - total recovered</b>", secondary_y=True)
fig.show()

In [208]:
dates = data['Date'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()
frame_pl = data[data['Province/State'] =="POLAND"]
frame_pl = frame_pl[frame_pl['Date']> pd.Timestamp(2020, 3, 6)]
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=dates, y=frame_pl['total_deaths'].values.tolist(), name="total_deaths"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=dates, y=frame_pl['total_cases'].values.tolist(), name="total_cases"),
    secondary_y=True,
)
fig.add_trace(
    go.Bar(x=dates, y=frame_pl['new_cases'].values.tolist(), name="new_cases"),
    secondary_y=False,
)
# Add figure title
fig.update_layout(
    title_text="Poland covid statistics"
)
# Set x-axis title
fig.update_xaxes(title_text="Date")
fig.update_layout(legend_orientation="v")
# Set y-axes titles
#fig.update_yaxes(title_text="<b>deaths - new cases</b>", secondary_y=False)
#fig.update_yaxes(title_text="<b>total cases - total recovered</b>", secondary_y=True)
fig.show()